In [1]:
import os
os.chdir("/home/kruu/git_folder/wake_encounter/wake_modeling")

from fwc_wake_encounter import main
import numpy as np
import pandas as pd

In [19]:
encounter_df = pd.read_parquet("/store/kruu/wake_encounter_simulations/t20_phi0_speed80_close_dist_theta_40_fixed_wake/encounters/0/encounter_df.parquet")

In [20]:
encounter_df

,X,Y,Z,hit_gate,dist_left_wake,dist_right_wake
Time,,,,,,
1.0,-1161.925554,-1039.037167,1973.0,False,961.817691,998.960486


In [2]:
def generate_encounter(v: float, # m/s, velocity of the aircraft
                       t_target: int, # s, time when the encounter hit the wake gate
                       time_range: int, # s, duration of the wakes
                       x_target: float, # m, target x-coordinate
                       y_target: float, # m, target y-coordinate
                       z_target: float, # m, target altitude-coordinate
                       theta: float, # Lateral angle (w.r.t. x_axis i.e. generator speed vector)
                       phi: float, # vertical angle (w.r.t. the y-axis)
                       ) -> pd.DataFrame:
    
    # Velocity components
    v_x = v * np.cos(theta)
    v_y = v * np.sin(theta) * np.cos(phi)
    v_z = v * np.sin(theta) * np.sin(phi)
    
    # Initial position
    x_0 = x_target - v_x * t_target
    y_0 = y_target - v_y * t_target
    z_0 = z_target - v_z * t_target
    
    # Time range of the wake simulation
    extended_times = np.arange(1, time_range + 1, step= 0.1) #Step to have refined timestamps for the trailer trajectory
    # extended_times = np.arange(1, time_range + 1)
    
    # Hit gate marker
    hit_mask = [False] * len(extended_times)
    hit_mask[t_target-1] = True
    
    # Positions for the extended time range
    positions = {
        'Time': extended_times,
        'X': x_0 + v_x * extended_times,
        'Y': y_0 + v_y * extended_times,
        'Z': z_0 + v_z * extended_times, 
        'hit_gate': hit_mask,
    }
    
    # Create the extended DataFrame
    encounter_trajectory = pd.DataFrame(positions).set_index('Time')
    
    return encounter_trajectory


In [3]:
generate_encounter(80, 20, 50, 0, 0, 0, 40, 0)

,X,Y,Z,hit_gate
Time,,,,
1.0,1013.745854,-1132.572004,0.0,False
1.1,1008.410349,-1126.611099,0.0,False
1.2,1003.074845,-1120.650193,0.0,False
1.3,997.739340,-1114.689288,0.0,False
1.4,992.403836,-1108.728383,0.0,False
...,...,...,...,...
50.5,-1627.328870,1818.076112,0.0,False
50.6,-1632.664375,1824.037017,0.0,False
50.7,-1637.999879,1829.997922,0.0,False


In [4]:
import subprocess

output_path = "/store/kruu/wake_encounter_simulations/t20_phi0_speed80_close_dist_theta_40_fixed_wake"
generator_path = "/home/kruu/git_folder/wake_encounter/wake_generation/generator.py"
fwc_path = "/home/kruu/git_folder/wake_encounter/wake_modeling/fwc_wake_encounter.py"
encounter_path = "/home/kruu/git_folder/wake_encounter/wake_generation/trailer.py"
id_run = 0

In [13]:
subprocess.run([
        "python", fwc_path, output_path, str(id_run),
        "--wake_id", "0", "--v_field", "True", "--typecode", "A320"
    ])

Encounter shape: (1, 4)
Wake shape: (1, 8)
time: -1
dir_t shape: (0, 3)
norms shape: (0, 1)
norms contents: []


/home/kruu/git_folder/wake_encounter/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/kruu/git_folder/wake_encounter/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Traceback (most recent call last):
  File "/home/kruu/git_folder/wake_encounter/wake_modeling/fwc_wake_encounter.py", line 134, in <module>
    main()
  File "/home/kruu/git_folder/wake_encounter/.venv/lib/python3.12/site-packages/click/core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/kruu/git_folder/wake_encounter/.venv/lib/python3.12/site-packages/click/core.py", line 1082, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/home/kruu/git_folder/wake_encounter/.venv/lib/python3.12/site-packages/click/core.py", l

CompletedProcess(args=['python', '/home/kruu/git_folder/wake_encounter/wake_modeling/fwc_wake_encounter.py', '/store/kruu/wake_encounter_simulations/t20_phi0_speed80_close_dist_theta_40_fixed_wake', '0', '--wake_id', '0', '--v_field', 'True', '--typecode', 'A320'], returncode=1)